In [ ]:
import numpy as np
import pandas as pd
import os, glob
from collections import Counter
from nltk.tokenize import wordpunct_tokenize as tokenize
import io, math, time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import random
import nltk

იმისათვის, რომ ჩავრვირთოთ ტორჩის მოდელი, საჭიროა რომ გავაკეთოთ მისი აღწერა:

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [ ]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


In [ ]:
train = []
for ind, filename in enumerate(glob.glob("../input/data-splitting/train/*")):
    if ind % 2000 == 0:
        print(ind, 'docs read')
    fd = io.open(filename, mode="r", encoding="utf-8")
    text = fd.read() 
    train.append(text)
    fd.close()
    
train = train[:35000]

train = [doc.replace('"', ' " ') for doc in train]
train_tokens = [tokenize(doc) for doc in train]
freqs = Counter()
for doc in train_tokens:
    freqs.update(doc)
top_words = freqs.most_common()[:50000]
vocab = ['UNK'] + [word for word, freq in top_words]

word_to_ind = {word:ind for ind, word in enumerate(vocab)}
ind_to_word = {ind:word for ind, word in enumerate(vocab)}


ჩავვირთოთ აწ უკვე დატრენინგებული მოდელი. მოდელი უკვე აღწერილია. საჭიროა მისი პარამეტრების დამთხვევა

In [ ]:
ntokens = 90001 # the size of vocabulary
emsize = 100 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.25 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)

model.load_state_dict(torch.load('../resources/pytorch-transformer-model/best_model.pth', map_location=torch.device('cpu')))


აღვწეროთ ფუნქცია რომელიც მოცემუ წინადადებას გაუკეთებს ტოკენიზაციას

In [ ]:
def tokenize_sent(sequence):
    """
    ფუნქცია რომელიც უკეთებს წინადადებას ტოკენიზაციას.
    Args: str: წინადადება
    Returns: აბრუნებს წინადადებების ტოკენების შესაბამის ინდექსებს
    """
    seq = sequence.replace('"', ' " ')
    seq = tokenize(seq)
    tokenized = []
    for word in seq:
        tokenized.append(torch.tensor(word_to_ind.get(word, 0)))

    data = torch.tensor(tokenized)
    data = data.view(1, -1).t().contiguous()
    return data


აღვწეროთ ფუნქცია, რომელიც გააგრძელებს მოცემულ წიანადადებას. ამ შემთხვევაში გაგრძელება მოხდება greedy და რანდომული არჩევანით.

In [ ]:
def continue_sent(sentence, seq_size):
    """
    Args: sentence: გასაგრძელებელი წინადედება
          seq_size: მაქსიმალური სიგრძე წინადადების
          n: 
    """
    seq = tokenize_sent(sentence)
    model.eval()
    while True:
        with torch.no_grad():
            if seq.size(0) == seq_size: break
            src_mask = model.generate_square_subsequent_mask(seq.size(0))
            output = model(seq, src_mask)
            output[-1][0][0] -= 500
            mx = torch.argmax(output[-1][0]).item()
            nxt = random.choices(list(range(90001)), output[-1][0].numpy())[0]
            print(ind_to_word[mx])
            seq = torch.cat((seq, torch.tensor(mx).view(1,1)), dim=0)
            
sentence = 'მსოფლიო საეკლესიო კრების'
continue_sent(sentence, 40)

ფუნქცია: რომელიც Beam Search-ით პულობს ყველაზე მაღალი შედეგის მქონდე N ცალ წინადადებას ხის pruning-ით 

In [ ]:
def continue_sent_beam(sentence, seq_size, n, max_len):
    """
    Args: sentence: გასაგრძელებელი წინადედება
          seq_size 
    """
    seq = tokenize_sent(sentence)
    seqs = [(seq, 0)] * n
    model.eval()
    k = 0
    while True:
        with torch.no_grad():
            # Pick top 2 items with highest score
            seqs = sorted(seqs, reverse=True, key=lambda x: x[1])[:n]
            print(seqs)
            for seq, score in seqs:
                if seq.size(0) == seq_size: break
                src_mask = model.generate_square_subsequent_mask(seq.size(0))
                output = model(seq, src_mask)
                output[-1][0][0] -= 500
                res, ind = torch.topk(output[-1][0], n)
                new_seqs = []
                for new_score, index in zip(res, ind):
                    new_seq = torch.cat((seq, torch.tensor(index).view(1,1)), dim=0)
                    new_seqs += [(new_seq, score+new_score)]
#                     seq = torch.cat((seq, torch.tensor(mx).view(1,1)), dim=0)
            seqs += new_seqs
        k += 1
        if k == max_len: break
    return seqs
            
sentence = 'მარა პრინციპში '
res = continue_sent_beam(sentence, 40, 2, 20)

1. ბეჭდავს Beam Search-ით მიღებული შედეგის პასუხს სტრინგად

In [ ]:
def seq_to_string(seq):
    res = ""
    for mx in seq[0]:
        print(mx)
        res +=  ind_to_word[mx.item()] + " "
    return res

seq_to_string(res[0])

ევალუციისთვის ვიყენებთ მარკეტერის სტატიებს. რომელიც სპეციალურად გაპარსა საბა სტურუამ. 

In [ ]:
bptt = 30

def evaluate(eval_model, data_source):
    eval_model.eval()
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)


წავიკითხოთ ტესტ-დატა

In [ ]:
test_data = pd.read_csv('../input/marketer-articles/articles.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_list = list(test_data['text'])
flattened_test_data = []
for text in data_list:
    if isinstance(text, str):
        sent_text = nltk.sent_tokenize(text)
        flattened_test_data += sent_text


In [ ]:
batch_size = 10
def batchify(data, bsz):
    nbatch = data.size(0) // bsz
    data = data.narrow(0, 0, nbatch * bsz)
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [ ]:
test_sequence = []
for sent in flattened_test_data:
    for word in nltk.word_tokenize(sent):
        test_sequence.append(torch.tensor(word_to_ind.get(word, 0)))
test_sequence = torch.tensor(test_sequence)
test_data = batchify(test_sequence, batch_size)


In [ ]:
criterion = nn.CrossEntropyLoss()
evaluation_result = evaluate(model, test_data)
